In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dianxin/train.csv
/kaggle/input/dianxin/test.csv
/kaggle/input/dianxin/sample_submit.csv


In [17]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

In [18]:
train = TabularDataset('/kaggle/input/dianxin/train.csv')
test = TabularDataset('/kaggle/input/dianxin/test.csv')

Loaded data from: /kaggle/input/dianxin/train.csv | Columns = 69 / 69 | Rows = 150000 -> 150000
Loaded data from: /kaggle/input/dianxin/test.csv | Columns = 68 / 68 | Rows = 30000 -> 30000


In [19]:
data = pd.concat([train, test], axis=0, ignore_index=True)
data

,客户ID,地理区域,是否双频,是否翻新机,当前手机价格,手机网络功能,婚姻状况,家庭成人人数,信息库匹配,预计收入,...,客户生命周期内平均月费用,客户生命周期内的平均每月使用分钟数,客户整个生命周期内的平均每月通话次数,过去三个月的平均每月使用分钟数,过去三个月的平均每月通话次数,过去三个月的平均月费用,过去六个月的平均每月使用分钟数,过去六个月的平均每月通话次数,过去六个月的平均月费用,是否流失
0,0,7,0,-1,181,0,2,0,0,3,...,24,286,91,351,121,23,303,101,25,0.0
1,1,13,1,0,1399,0,3,0,0,0,...,44,447,190,483,199,40,488,202,44,1.0
2,2,14,1,0,927,0,2,4,0,6,...,48,183,79,271,95,71,209,77,54,0.0
3,3,1,0,0,232,0,3,-1,1,-1,...,42,303,166,473,226,72,446,219,65,1.0
4,4,0,-1,0,699,0,1,2,0,3,...,36,119,24,88,15,35,106,21,37,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,179995,4,0,0,173,1,2,2,0,2,...,49,309,95,457,163,97,377,136,73,NaN
179996,179996,9,0,0,150,1,1,3,0,8,...,37,106,66,52,12,28,46,15,28,NaN
179997,179997,6,1,0,1049,0,1,3,0,6,...,37,299,179,243,143,39,300,179,38,NaN
179998,179998,16,0,0,209,1,-1,-1,1,-1,...,22,58,30,12,8,13,33,16,19,NaN


In [20]:
features = [f for f in data.columns if f not in ['是否流失','客户ID']]

train = data[data['是否流失'].notnull()].reset_index(drop=True)
test = data[data['是否流失'].isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['是否流失']

# 3. 两两组合

In [21]:
from sklearn.decomposition import PCA

In [22]:
# 训练集pca
train_multi=pd.DataFrame()
for i in range(x_train.shape[1]-1):
	train_multi=pd.concat([train_multi,pd.DataFrame(np.array(x_train.iloc[:,i])+
	np.array(x_train.iloc[:,(i+1):x_train.shape[1]]).T)],axis=0) 

train_multi=train_multi.T

In [23]:
train_pca = PCA(n_components=10, whiten=True).fit(train_multi) # 降维到2D
X_train_pca = train_pca.transform(train_multi)
X_train_pca

array([[-0.57492426,  0.03516996, -0.24077646, ..., -0.78135387,
         0.0812872 , -0.44607325],
       [-0.34019489,  0.22969909, -0.71143511, ...,  0.08319503,
        -0.39023442, -0.28738895],
       [-0.33460017,  0.27520337,  0.54154411, ..., -0.78165951,
        -0.10686904, -0.0219435 ],
       ...,
       [ 1.54299566,  0.16476596,  0.66404451, ..., -1.12171296,
         0.18696817, -0.26305028],
       [ 0.22430528, -0.11903477,  0.30964276, ...,  1.57955373,
         0.8033411 , -0.84709312],
       [-0.62793215,  0.27520841,  0.08913223, ..., -0.176742  ,
        -0.14440193, -0.25049195]])

In [24]:
# 测试集pca
test_multi=pd.DataFrame()
for i in range(x_test.shape[1]-1):
	test_multi=pd.concat([test_multi,pd.DataFrame(np.array(x_test.iloc[:,i])+
	np.array(x_test.iloc[:,(i+1):x_test.shape[1]]).T)],axis=0) 

test_multi=test_multi.T

In [25]:
test_multi

,0,1,2,3,4,5,6,7,8,9,...,0,1,2,3,0,1,2,0,1,0
0,16,15,924,15,18,14,16,14,16,442,...,80,180,98,79,182,100,81,200,181,99
1,9,8,1181,8,9,9,8,13,8,66,...,397,1369,630,399,1156,417,186,1389,1158,419
2,8,7,1056,7,9,7,7,9,7,420,...,299,805,433,300,662,290,157,796,663,291
3,-1,-1,208,0,-1,0,-1,6,-1,599,...,86,129,107,86,109,87,66,130,109,87
4,-1,-1,208,1,-2,-2,0,-2,-2,1202,...,55,105,74,53,88,57,36,107,86,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,4,4,177,5,6,6,4,6,4,506,...,260,540,299,236,474,233,170,513,450,209
29996,9,9,159,10,10,12,9,17,9,386,...,40,58,27,40,74,43,56,61,74,43
29997,7,6,1055,6,7,9,6,12,6,212,...,182,443,322,181,339,218,77,479,338,217
29998,16,16,225,17,15,15,17,15,15,699,...,21,41,24,27,46,29,32,49,52,35


In [26]:
test_pca = PCA(n_components=10, whiten=True).fit(test_multi) # 降维到2D
X_test_pca = test_pca.transform(test_multi)
X_test_pca

array([[ 0.13396737,  0.2333674 , -1.72235356, ...,  0.57164638,
        -0.0667092 ,  0.25060715],
       [ 1.00575593, -0.88416492,  0.19218228, ...,  0.49078108,
        -1.03003328,  0.07811813],
       [-0.14429613,  0.03568047,  0.52856342, ..., -0.43368621,
        -0.05050301, -0.09439974],
       ...,
       [-0.65587528,  0.44250759,  0.46832549, ...,  0.00634285,
         0.18243222, -0.91878089],
       [-0.73684968,  0.32435153, -0.61957964, ...,  0.0243409 ,
        -0.1624039 , -0.01995731],
       [ 0.97287787, -2.42904341, -0.36970941, ..., -1.3449205 ,
        -0.97967944, -0.28991003]])

In [27]:
X_train=pd.DataFrame(X_train_pca)
X_test=pd.DataFrame(X_test_pca)

# 4. 构建模型

In [28]:
X_train['label'] = y_train

In [29]:
X_train

,0,1,2,3,4,5,6,7,8,9,label
0,-0.574924,0.035170,-0.240776,0.788226,-1.238421,-0.260969,0.860680,-0.781354,0.081287,-0.446073,0.0
1,-0.340195,0.229699,-0.711435,0.240014,1.631456,0.274230,0.244690,0.083195,-0.390234,-0.287389,1.0
2,-0.334600,0.275203,0.541544,-0.977777,0.863193,0.122358,0.428353,-0.781660,-0.106869,-0.021944,0.0
3,-0.002340,0.878244,0.081193,-0.300616,-1.310670,-0.033621,-1.038054,-0.377561,-0.504660,-0.301621,1.0
4,-0.747318,0.049257,0.314851,0.520952,0.258622,-0.256990,0.404904,-0.240403,-0.331033,0.021547,1.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0.349709,-0.364457,2.556410,-4.283498,2.603404,0.440117,1.123692,2.404083,-0.239129,1.224989,1.0
149996,1.210171,-2.524345,-0.264913,2.389323,-0.759814,-1.519220,-0.160070,-0.056304,-1.128181,-1.004814,0.0
149997,1.542996,0.164766,0.664045,1.843583,1.474261,0.587155,-1.467036,-1.121713,0.186968,-0.263050,1.0
149998,0.224305,-0.119035,0.309643,-0.024934,2.096273,1.512722,1.078488,1.579554,0.803341,-0.847093,1.0


In [30]:
predictor = TabularPredictor(label='label').fit(train_data=X_train)
predictions = predictor.predict(X_test)

No path specified. Models will be saved in: "AutogluonModels/ag-20221012_011509/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221012_011509/"
AutoGluon Version:  0.5.2
Python Version:     3.7.12
Operating System:   Linux
Train Data Rows:    150000
Train Data Columns: 10
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess 

	0.6296	 = Validation score   (accuracy)
	12.58s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: LightGBM ...
	0.6268	 = Validation score   (accuracy)
	3.47s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7576	 = Validation score   (accuracy)
	97.51s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7648	 = Validation score   (accuracy)
	138.98s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: CatBoost ...
	0.6164	 = Validation score   (accuracy)
	12.93s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7344	 = Validation score   (accuracy)
	21.81s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7384	 = Validation score   (accuracy)
	21.53s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.622	 = Validation score   (accuracy)
	181.21s	 = Training   runtim

In [31]:
test['是否流失'] = predictions
test[['客户ID', '是否流失']].to_csv('autogloun.csv', index=False)